In [1]:
import jarvis
import pandas as pd
import numpy as np

# Where do I start? Summarize current and past experiments

In [2]:
summary = jarvis.listVersionSummaries('twitter_demo')

100%|██████████| 2/2 [00:12<00:00,  6.16s/it]


In [3]:
summary.reset_index(drop=True)

,__trialNum__,alpha,frac,model,model_accuracy,split_seed,__commitHash__
0,9,0.9,0.75,NOT LOADED,0.58152,42,c01502400e82d88fffa8a1988085fa04421e2dd2
1,0,0.0,0.75,NOT LOADED,0.79514,42,c01502400e82d88fffa8a1988085fa04421e2dd2
2,7,0.7,0.75,NOT LOADED,0.60018,42,c01502400e82d88fffa8a1988085fa04421e2dd2
3,6,0.6,0.75,NOT LOADED,0.61348,42,c01502400e82d88fffa8a1988085fa04421e2dd2
4,1,0.1,0.75,NOT LOADED,0.73202,42,c01502400e82d88fffa8a1988085fa04421e2dd2
5,10,1.0,0.75,NOT LOADED,0.57407,42,c01502400e82d88fffa8a1988085fa04421e2dd2
6,8,0.8,0.75,NOT LOADED,0.58993,42,c01502400e82d88fffa8a1988085fa04421e2dd2
7,4,0.4,0.75,NOT LOADED,0.64544,42,c01502400e82d88fffa8a1988085fa04421e2dd2
8,3,0.3,0.75,NOT LOADED,0.66530,42,c01502400e82d88fffa8a1988085fa04421e2dd2
9,2,0.2,0.75,NOT LOADED,0.69506,42,c01502400e82d88fffa8a1988085fa04421e2dd2


# What is the best experiment?
Here, we measure _goodness_ of an experiment by its average model accuracy, over all trials.

In [4]:
summary_stat = summary.groupby('__commitHash__')['model_accuracy'].mean()
summary_stat

__commitHash__
597cb760c89b1db5b3efd26c981539fffe79bb83    0.713991
c01502400e82d88fffa8a1988085fa04421e2dd2    0.647153
Name: model_accuracy, dtype: float64

In [5]:
best_commit_hash = summary_stat.idxmax()
best_commit_hash

'597cb760c89b1db5b3efd26c981539fffe79bb83'

# What is the best trial, given the best experiment?

In [13]:
best_index = summary.loc[summary['__commitHash__'] == best_commit_hash]['model_accuracy'].idxmax()
best_trial = summary.iloc[best_index]['__trialNum__']
best_trial

'0'

# So the former experiment is better than the latter. What changed?

In [16]:
versions = summary.__commitHash__.unique()
most_recent, least_recent = versions

In [17]:
jarvis.diffExperimentVersions('twitter_demo', least_recent, most_recent)

train_model.py --> train_model.py
@@ -47,8 +47,8 @@ def train(tweet_df, alpha):
     ## Convert tweet to bag of words for learning
 
     # Tokenize Text
-    count_vect = CountVectorizer()
-    #count_vect = TfidfVectorizer()
+    #count_vect = CountVectorizer()
+    count_vect = TfidfVectorizer()
     X_train = count_vect.fit_transform(tweet_df["tweet"])
 
     intermediary["vectorizer"] = count_vect


# Let's get the best model yet!
From the best trial, from the best experiment

In [18]:
best_intermediary = jarvis.materialize('twitter_demo', best_trial, best_commit_hash, 'intermediary.pkl')

In [19]:
best_intermediary

{'classifier': MultinomialNB(alpha=0.0, class_prior=None, fit_prior=True),
 'country_dict': {' ': 32,
  ' AD': 21,
  ' AE': 105,
  ' AF': 15,
  ' AG': 119,
  ' AI': 106,
  ' AL': 155,
  ' AM': 148,
  ' AO': 25,
  ' AR': 11,
  ' AT': 62,
  ' AU': 55,
  ' AW': 66,
  ' AZ': 77,
  ' BA': 92,
  ' BB': 79,
  ' BD': 28,
  ' BE': 149,
  ' BG': 29,
  ' BH': 157,
  ' BI': 150,
  ' BJ': 12,
  ' BM': 98,
  ' BN': 18,
  ' BO': 53,
  ' BQ': 156,
  ' BR': 97,
  ' BS': 9,
  ' BW': 91,
  ' BY': 7,
  ' CA': 50,
  ' CD': 162,
  ' CG': 90,
  ' CH': 59,
  ' CI': 40,
  ' CL': 22,
  ' CM': 121,
  ' CN': 36,
  ' CO': 100,
  ' CR': 76,
  ' CU': 102,
  ' CV': 96,
  ' CY': 74,
  ' CZ': 116,
  ' DE': 169,
  ' DK': 34,
  ' DM': 109,
  ' DO': 137,
  ' DZ': 115,
  ' EC': 111,
  ' EE': 80,
  ' EG': 85,
  ' ES': 163,
  ' ET': 142,
  ' FI': 67,
  ' FR': 78,
  ' GA': 112,
  ' GB': 30,
  ' GE': 138,
  ' GH': 145,
  ' GI': 56,
  ' GL': 173,
  ' GN': 108,
  ' GP': 143,
  ' GR': 172,
  ' GT': 19,
  ' GU': 14,
  ' GY': 63,
 

# Here's the best model in action

In [20]:
country_dict = best_intermediary['country_dict']
classifier = best_intermediary['classifier']
vectorizer = best_intermediary['vectorizer']

code_dict = {}

for kee in country_dict:
    code_dict[country_dict[kee]] = kee

while True:
    tweet = input("What's on your mind? ")
    if tweet == 'exit':
        break
    tweet_vec = vectorizer.transform(np.array([tweet,]))
    country_id = classifier.predict(tweet_vec)
    print("Predicted country of origin: {}\n".format(code_dict[country_id[0]]))

What's on your mind? OMG this works!! :D
Predicted country of origin:  US

What's on your mind? exit
